In [ ]:
import geojson
from geopy.distance import geodesic

### This file cuts a GeoJSON at the closest point to a chosen lat/lng and discards the remainder, REPLACING the original file

### Use with caution

In [ ]:
# Define these CAREFULLY!! - It will REPLACE the original file

################################
# path to geojson
# file_path = '../docs/resources/geojson/boat_pacuare.geojson'

# set coordinates to cut the trail / road / track
# Note: (latitude, longitude)
# endpoint = (10.098051, -83.483936)
#################################

In [ ]:
# functions for cutting a hiking trail / road / train track

# find the closest point to the endpoint on the trail / road / track
def find_closest_point(feature, endpoint):
    closest_point = None
    min_distance = float("inf")
    for coord in feature["geometry"]["coordinates"]:
        # convert to (latitude, longitude) for geodesic calculation
        coord_latlon = (coord[1], coord[0])
        distance = geodesic(coord_latlon, endpoint).meters
        if distance < min_distance:
            min_distance = distance
            closest_point = coord
    return closest_point

# filter the trail up to the closest point
def filter_trail(feature, closest_point):
    filtered_coords = []
    for coord in feature["geometry"]["coordinates"]:
        filtered_coords.append(coord)
        if coord == closest_point:  # stop at the closest point
            break
    return filtered_coords

In [ ]:
# loads, processes, and saves the geojson file

# load geojson
with open(file_path, "r") as file:
    data = geojson.load(file)

# process geojson
filtered_features = []
for feature in data["features"]:
    if feature["geometry"]["type"] == "LineString":
        # find the closest point to the endpoint
        closest_point = find_closest_point(feature, endpoint)
        if closest_point:
            # filter to the closest point
            filtered_coords = filter_trail(feature, closest_point)
            if filtered_coords:
                # create new feature with the filtered geometry
                new_feature = geojson.Feature(
                    geometry=geojson.LineString(filtered_coords),
                    properties=feature["properties"],
                )
                filtered_features.append(new_feature)

# save
filtered_geojson = geojson.FeatureCollection(filtered_features)
with open(file_path, "w") as file:
    geojson.dump(filtered_geojson, file, indent=2)

print("Filtered GeoJSON saved!")

Filtered GeoJSON saved!
